In [15]:
import pandas as pd 
import numpy as np 
import os
import json
from dataclasses import dataclass


In [16]:
url = "/home/flavio/OneDrive/NLeSC/Projects/2024_life2vec/stakeholders/CBS/exports/2024-05-14/data/llm"

In [17]:
def yield_files(root):
    for root, dirs, files in os.walk(root):
        for filename in files:
            filename = os.path.join(root, filename)
            if os.path.isfile(filename):  #or os.path.isdir(filename):  
                yield filename          
                # yield FileInfo(filename)

In [18]:
files = list(yield_files(url))

In [19]:
files

['/home/flavio/OneDrive/NLeSC/Projects/2024_life2vec/stakeholders/CBS/exports/2024-05-14/data/llm/jobs_full_duration_meta.txt',
 '/home/flavio/OneDrive/NLeSC/Projects/2024_life2vec/stakeholders/CBS/exports/2024-05-14/data/llm/.~lock.new_death_info_good_format_columns.csv#',
 '/home/flavio/OneDrive/NLeSC/Projects/2024_life2vec/stakeholders/CBS/exports/2024-05-14/data/llm/divorce_info_good_format_covariance.csv',
 '/home/flavio/OneDrive/NLeSC/Projects/2024_life2vec/stakeholders/CBS/exports/2024-05-14/data/llm/education_info_good_format_covariance.csv',
 '/home/flavio/OneDrive/NLeSC/Projects/2024_life2vec/stakeholders/CBS/exports/2024-05-14/data/llm/background_columns.csv',
 '/home/flavio/OneDrive/NLeSC/Projects/2024_life2vec/stakeholders/CBS/exports/2024-05-14/data/llm/background_meta.txt',
 '/home/flavio/OneDrive/NLeSC/Projects/2024_life2vec/stakeholders/CBS/exports/2024-05-14/data/llm/education_info_good_format_columns.csv',
 '/home/flavio/OneDrive/NLeSC/Projects/2024_life2vec/stakehol

Notes
- this extracts all files
- but we want only the unique file names "path/filename.x". Ie, strip off the "columns", "covariance", "meta" from the file name
    - but we need this information to generate the fake data
- thus, read in the unique file names, store the data from the three files (in a data class?)
- data class vs function?
    - dataclass nicer to work with than dict 
    - we can still create a function that generates the fake data and takes the data class as an input?
- can we make this generalize to other data (the raw CBS data?) 
- also write tests for this?


categorical variables can also be stored in integers. when should such a variable be generated as categorical?
- problem is that we don't know the class probabilities 
- what is the cost of erring on one side or the other?
    - is categorical but assume numerical -> 
    - is numerical but assume categorical 
- simple rule: if the q10 and q90 are below QMAX, assume categorical with equal distribution across classes 
- for municipality/age
    - set QMAX = 10?
    - I guess a more general rule is that there are not more than QMAX distinct integers between q10 and q90
    - it is very unlikely to match q10 and q90 with a normal, given the mean and std 
    - we can fix this by adding some equal-weighted categoricals in some range [q10, x1] and [x2, q90]. 
    - how to determine x1 and x2? by chance, we may get min(generated) < q10, in which case nothing would happen
        - instead, use x1 = q10(generated) and x2 = q90(generated)?
    - for municipality, there will be negative draws -> replace them with 0, 1, or q10
- for RINPERSOON (and other panel variables/PII variables): 
    - define base set of identifiers (1-100), then draw with random_choice
        - this would be the preferred way, but we don't know if each row of PII variables are unique in a given dataframe or not

In [20]:
def detect_variable_type(row, max_diff_q10_q90=10):
    """Detect how a fake variable should be generated.

    A column is detected as categorical if one of two conditions hold
    - it has at least one category reported
    - it is no category reported, but the summary statistics
      imply only a limited number of possible classes.

    Args:
        row (dict): a row from a pd.DataFrame
        max_diff_q10_q90 (int, optional): The maximum difference 
        between the 10th and 90th percentile in the empirical distribution.
        This is used to infer the second case of categorical variables.

    Returns:
        dict: instructions to generate random variables.
    
    """
    category_0 = row["category_top_0"]
    if isinstance(category_0, str):
        top_cats = [row[f"category_top_{i}"] for i in range(5)]
        top_cats = [x for x in top_cats if isinstance(x, str)]
        top_cats = [x.split("--") for x in top_cats]
        classes = [x[0] for x in top_cats]
        probs = [float(x[1]) for x in top_cats]
        result_dict = {
            "type": "categorical",
            "classes": classes,
            "probs": probs
        }
    else:
        p10, p90 = row["10th_percentile"], row["90th_percentile"]
        if p90 - p10 <= max_diff_q10_q90:
            # assert isinstance(p10, int) and isinstance(p90, int), "assuming integers which is false"
            pdiff = int(p90) - int(p10)
            # we want to include the end of the range, and deal with cases where p10=p90
            addon = 1 if pdiff > 0 else 2
            p90 += addon
            pdiff += addon
            result_dict = {
                "type": "categorical",
                "classes": np.arange(int(p10), int(p90)),
                "probs": [1/pdiff for _ in range(pdiff)]
            } 
        else:
            result_dict = {
                "type": "continuous",
                "mean": row["mean"],
                "std_dev": row["std_dev"],
                "min": p10
            }

    result_dict["null_fraction"] = row["null_fraction"]
    
    return result_dict
    
# null fraction! 


In [21]:
files = os.walk(url)
for f in files:
    print(f)

('/home/flavio/OneDrive/NLeSC/Projects/2024_life2vec/stakeholders/CBS/exports/2024-05-14/data/llm', ['income_yearly', 'job_yearly'], ['jobs_full_duration_meta.txt', '.~lock.new_death_info_good_format_columns.csv#', 'divorce_info_good_format_covariance.csv', 'education_info_good_format_covariance.csv', 'background_columns.csv', 'background_meta.txt', 'education_info_good_format_columns.csv', 'new_death_info_good_format_meta.txt', 'divorce_info_good_format_meta.txt', 'new_death_info_good_format_columns.csv', 'jobs_full_duration_covariance.csv', 'new_death_info_good_format_covariance.csv', 'divorce_info_good_format_columns.csv', 'jobs_full_duration_columns.csv', 'background_covariance.csv', 'education_info_good_format_meta.txt'])
('/home/flavio/OneDrive/NLeSC/Projects/2024_life2vec/stakeholders/CBS/exports/2024-05-14/data/llm/income_yearly', [], ['income_17_columns.csv', 'income_16_columns.csv', 'income_15_covariance.csv', 'income_18_covariance.csv', 'income_20_meta.txt', 'income_21_colum

This is the main data class we are working with -- **give it a better name**

In [22]:
@dataclass
class DataSummary: # TODO: give better name
    """Class to read summary files from CBS.

    Arguments:
        url: local path where the summary files are stored
        filename: the name of the original data file 
    """
    url: str 
    filename: str 


    def load(self):
        "Load meta data and column summaries"
        with open(os.path.join(self.url, self.filename + "_meta.txt"), "r") as f:
            self.meta = dict(json.load(f))
        
        self.col_summary = pd.read_csv(os.path.join(self.url, self.filename + "_columns.csv"))


    def fit(self):
        """Process metadata and summary statistics to define how each column should be generated.
        """
        results = {}
        for _, row in self.col_summary.iterrows():
            variable = row["variable_name"]
            results[variable] = detect_variable_type(row)
        
        self.generation_inputs = results


    def generate(self, rng, size=None):
        """Generate new data while
            - enforcing non-negativity of numerical variables
            - considering PII columns: each column is an integer range from 0 to `size`. 
            - considering null fractions
        
        Args:
            rng (np.random.default_rng): random number generator
            n (int, optional): Size of sample to generate. If `None`, the size is taken from
            the data summary.

        Returns:
            pd.DataFrame: the generated data
        """
        assert self.generation_inputs, "You need to `fit` before calling `generate`."
        if size is None:
            size = self.meta.get("total_nobs")
        column_dtypes = self.meta.get("columns_with_dtypes")

        data = {}

        pii_cols = self.meta.get("has_pii_columns")
        for pc in pii_cols:
            data[pc] = np.arange(size)

        for colname, inputs in self.generation_inputs.items(): 
            # TODO -- put this check into a function      
            required_dtype = column_dtypes.get(colname)
            implemented_types = ["object", "int64"]
            if required_dtype not in implemented_types:
                raise NotImplementedError("data type %s not implemented" % required_dtype)
        
            n_nulls = int(size * inputs["null_fraction"])
            n_nonulls = size - n_nulls 

            if inputs["type"] == "categorical":
                col_data = rng.choice(a=inputs["classes"], size=n_nonulls, p=inputs["probs"])
            elif inputs["type"] == "continuous":
                col_data = rng.normal(inputs["mean"], inputs["std_dev"], n_nonulls)
                min_value = inputs["min"]
                negatives = np.where(col_data < 0)
                col_data[negatives] = min_value
                if required_dtype == "int64":
                    col_data = np.int64(np.round(col_data))

            nulls = np.tile(np.nan, n_nulls)
            col_data = np.concatenate([nulls, col_data])
            rng.shuffle(col_data)

            data[colname] = col_data

        return pd.DataFrame(data)
        


In [23]:
jobs_full_duration = DataSummary(url=url, filename="jobs_full_duration")
jobs_full_duration.load()

In [24]:
jobs_full_duration.meta

{'path': '/gpfs/ostor/ossc9424/homedir/Tanzir/LifeToVec_Nov/projects//dutch_real/data/jobs_full_duration.csv',
 'shape': [27288936, 11],
 'columns_with_dtypes': {'RINPERSOON': 'int64',
  'age': 'int64',
  'daysSinceFirstEvent': 'int64',
  'contractType2': 'int64',
  'sicknessInsurance2': 'int64',
  'fullTime2': 'int64',
  'sector2': 'int64',
  'industry2': 'int64',
  'jobType2': 'int64',
  'empRelationship2': 'int64',
  'begOrEnd': 'int64'},
 'total_nobs': 27288936,
 'nobs_sumstat': None,
 'has_pii_columns': ['RINPERSOON']}

In [25]:
jobs_full_duration.col_summary

,variable_name,median,mean,std_dev,10th_percentile,90th_percentile,q1,q3,null_fraction,category_top_0,category_top_1,category_top_2,category_top_3,category_top_4,_others
0,age,28.0,28.692786,5.121164,22.0,36.0,25.0,32.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,daysSinceFirstEvent,16076.0,16261.625102,1452.578314,14397.0,18507.0,14977.0,17472.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,contractType2,1.0,1.685221,0.945901,1.0,3.0,1.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,sicknessInsurance2,1.0,1.036220,0.186836,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,fullTime2,2.0,1.593243,0.491229,1.0,2.0,1.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5,sector2,1.0,1.486110,1.164521,1.0,2.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6,industry2,36.0,30.494275,20.903393,1.0,55.0,10.0,46.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7,jobType2,5.0,4.672533,1.054801,5.0,5.0,5.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
8,empRelationship2,1.0,3.748129,5.093709,1.0,11.0,1.0,7.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
9,begOrEnd,1.5,1.500000,0.500000,1.0,2.0,1.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
jobs_full_duration.fit()

In [27]:
my_rng = np.random.default_rng(seed=99303)

In [28]:
data = jobs_full_duration.generate(rng=my_rng, size=1_000)

In [29]:
data.head()
data.describe()

,RINPERSOON,age,daysSinceFirstEvent,contractType2,sicknessInsurance2,fullTime2,sector2,industry2,jobType2,empRelationship2,begOrEnd
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,28.579000,16204.472000,1.989000,1.513000,1.495000,1.497000,31.307000,5.521000,5.981000,1.483000
std,288.819436,5.240727,1490.877172,0.813351,0.500081,0.500225,0.500241,19.524906,0.499809,3.130565,0.499961
min,0.000000,12.000000,11691.000000,1.000000,1.000000,1.000000,1.000000,0.000000,5.000000,1.000000,1.000000
25%,249.750000,25.000000,15212.000000,1.000000,1.000000,1.000000,1.000000,16.750000,5.000000,3.000000,1.000000
50%,499.500000,29.000000,16162.000000,2.000000,2.000000,1.000000,1.000000,31.000000,6.000000,6.000000,1.000000
75%,749.250000,32.000000,17262.000000,3.000000,2.000000,2.000000,2.000000,44.000000,6.000000,9.000000,2.000000
max,999.000000,46.000000,20660.000000,3.000000,2.000000,2.000000,2.000000,110.000000,6.000000,11.000000,2.000000


## We need to iterate the above across all files -- tbd
- also, for the raw CBS files, we need to define which years we want to generate the data for

In [34]:
current_file = "new_death_info_good_format"
meta = current_file + "_meta.txt"
covariance = current_file + "_covariance.csv"
columns = current_file + "_columns.csv"

In [35]:
with open(os.path.join(url, meta), "r") as f:
    metadata = dict(json.load(f))

In [36]:
metadata

{'path': '/gpfs/ostor/ossc9424/homedir/Tanzir/LifeToVec_Nov/projects//dutch_real/data/new_death_info_good_format.csv',
 'shape': [4316610, 4],
 'columns_with_dtypes': {'RINPERSOON': 'int64',
  'daysSinceFirstEvent': 'int64',
  'age': 'int64',
  '[DEATH]': 'object'},
 'total_nobs': 4316610,
 'nobs_sumstat': None,
 'has_pii_columns': ['RINPERSOON']}

In [37]:
d_cols = pd.read_csv(os.path.join(url, columns))
d_cols.head()

,variable_name,median,mean,std_dev,10th_percentile,90th_percentile,q1,q3,null_fraction,category_top_0,category_top_1,category_top_2,category_top_3,category_top_4,_others
0,daysSinceFirstEvent,13992.0,1.384631e+04,3.096643e+03,9462.9,17969.0,11133.0,16607.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,age,80.0,-5.769135e+14,7.294350e+16,57.0,92.0,70.0,87.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,[DEATH],NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,D--1.0,EMPTY--0.0,NaN,NaN,NaN,0.0


In [63]:
int(1.0) == 1.0
isinstance(1.0, np.float64)

False

In [21]:
rng = np.random.default_rng(seed=5958375235)

In [23]:
testdraws = np.round(rng.normal(loc=345, scale=328, size=100_000))
print(testdraws.min())
print(testdraws.max())

-1258.0
1846.0


In [24]:
np.nanpercentile(testdraws, [1, 99])

array([-418., 1111.])

In [10]:
?rng.choice

Docstring:
choice(a, size=None, replace=True, p=None, axis=0, shuffle=True)

Generates a random sample from a given array

Parameters
----------
a : {array_like, int}
    If an ndarray, a random sample is generated from its elements.
    If an int, the random sample is generated from np.arange(a).
size : {int, tuple[int]}, optional
    Output shape.  If the given shape is, e.g., ``(m, n, k)``, then
    ``m * n * k`` samples are drawn from the 1-d `a`. If `a` has more
    than one dimension, the `size` shape will be inserted into the
    `axis` dimension, so the output ``ndim`` will be ``a.ndim - 1 +
    len(size)``. Default is None, in which case a single value is
    returned.
replace : bool, optional
    Whether the sample is with or without replacement. Default is True,
    meaning that a value of ``a`` can be selected multiple times.
p : 1-D array_like, optional
    The probabilities associated with each entry in a.
    If not given, the sample assumes a uniform distribution over a

In [11]:
?rng.normal

Docstring:
normal(loc=0.0, scale=1.0, size=None)

Draw random samples from a normal (Gaussian) distribution.

The probability density function of the normal distribution, first
derived by De Moivre and 200 years later by both Gauss and Laplace
independently [2]_, is often called the bell curve because of
its characteristic shape (see the example below).

The normal distributions occurs often in nature.  For example, it
describes the commonly occurring distribution of samples influenced
by a large number of tiny, random disturbances, each with its own
unique distribution [2]_.

Parameters
----------
loc : float or array_like of floats
    Mean ("centre") of the distribution.
scale : float or array_like of floats
    Standard deviation (spread or "width") of the distribution. Must be
    non-negative.
size : int or tuple of ints, optional
    Output shape.  If the given shape is, e.g., ``(m, n, k)``, then
    ``m * n * k`` samples are drawn.  If size is ``None`` (default),
    a single v